In [200]:
import os
import sys
sys.path.insert(0,'..')
import pandas as pd
from lib_template.settings import DATA_DIR
import glob
import pathlib
import joblib
import pickle


def _process_data(DATA_DIR, temporary_dir= "temporary", processed_dir= "persistent", file_type= "csv"):
    file_list = glob.glob(f'{DATA_DIR}\\{temporary_dir}\\**', recursive=True)
    file_of_matched_type = [file for file in file_list if file.endswith(f".{file_type}")]
    matched_dfs = [pd.read_csv(file) for file in file_of_matched_type]
    file_hash = [joblib.hash(df, hash_name='sha1') for df in matched_dfs]
    file_names = [str(pathlib.Path(f).name).replace(f".{file_type}","") for f in file_of_matched_type]
    new_files_collector = []
    for hash_key, old_file in zip(file_hash,file_of_matched_type):
        new_file_name = f"{hash_key}.{file_type}"
        new_file = pathlib.Path(os.path.join(DATA_DIR, processed_dir, new_file_name))
        old_file_path = pathlib.Path(old_file)
        if not new_file.is_file():
            old_file_path.rename(new_file)
        old_file_path.unlink(missing_ok = True)
        new_files_collector.append(new_file)
    data_lock = dict(zip(file_hash, new_files_collector))
    data_name_dict = dict(zip(file_hash, file_names))
    return data_lock, data_name_dict


def _generate_data_lock(DATA_DIR, data_hash_lock="local_data_lock.pickle", data_name_metadata = "local_meta_data.pickle"):
    new_data_lock, data_name_dict = _process_data(DATA_DIR)
    data_lock_path = pathlib.Path(os.path.join(DATA_DIR, data_hash_lock))
    meta_data_path = pathlib.Path(os.path.join(DATA_DIR, data_name_metadata))
    if data_lock_path.is_file():
        with open(data_lock_path, 'rb') as handle:
            old_data_lock = pickle.load(handle)
        if new_data_lock:
            data_lock = {**old_data_lock, **new_data_lock}
        else:
            data_lock = old_data_lock.copy()
    else:
        data_lock = new_data_lock.copy()
    if data_lock:
        with open(data_lock_path, 'wb') as handle:
            pickle.dump(data_lock, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return data_lock, data_lock_path

In [201]:
data_lock, data_lock_path = _generate_data_lock(DATA_DIR)

In [202]:
data_lock_path

WindowsPath('C:/Workspace/projects/lib_template/data/local_data_lock.pickle')

In [203]:
data_lock

{'090ade65772b70f36184e84745715a0d6c277982': WindowsPath('C:/Workspace/projects/lib_template/data/persistent/090ade65772b70f36184e84745715a0d6c277982.csv'),
 'c0e894ee39909eb2cc05fd9f597b016a908fcb5d': WindowsPath('C:/Workspace/projects/lib_template/data/persistent/c0e894ee39909eb2cc05fd9f597b016a908fcb5d.csv')}

In [185]:
# get list of files present in persistent folder in blob.
# whatever is not matching with the local data_lock file upload that
# update the online data lock file


# read the file present in presistent folder. extract hash key of each file.
# read data lock file in local. check diff, download only the files which are not present in local.
# update the local data lock file.